# 划分数据集

## 1. 导入工具包

In [17]:
# 导入操作系统库以获取平台信息以及对目录操作的权限
import os

# 导入正则表达式库以支持字符串的分割操作
import re

# 导入进度条库的模块以令进程可视化
from tqdm import tqdm

# 导入文件操作库以支持os模块所没有的高级文件操作
import shutil

# 导入随机函数库以支持数据集的随机划分操作
import random

# 导入数据分析库以支持结构化数据分析
import pandas

## 2. 创建训练集文件夹和验证集文件夹

In [23]:
'''
re的split函数用于根据正则表达式匹配将字符串进行分割并以列表形式返回，构造方法如下：
re.split(pattern, string, maxsplit = 0, flags = 0)
pattern : 相当于str.split()中的sep，分隔符的意思，可以是字符串，也可以为正则表达式（例：'[ab]'表示取a和b的任意一个值）
string : 要进行分割的字符串
maxsplit : 分割的最大次数，默认为0即不限次数（注意和str.split()中的有点不一样）
flags : 修改pattern表达式的功能（比如忽略大小写就用re.IGNORECASE）
'''

# 指定工作目录
WorkDir = 'DataSet' # 指定数据集文件夹为工作目录

# 先创建训练集文件夹与验证集文件夹
if not os.path.exists(WorkDir + '/Training') and not os.path.exists(WorkDir + '/Validation'): # 防止因重复创建而报错
    os.mkdir(os.path.join(WorkDir, 'Training'), mode = 0o777) # Path: %WorkDir%/Training
    os.mkdir(os.path.join(WorkDir, 'Validation'), mode = 0o777) # Path: %Workdir%/Validation
else:
    try:
        os.mkdir(os.path.join(WorkDir, 'Training'), mode = 0o777) # Path: %WorkDir%/Training
    except:
        os.mkdir(os.path.join(WorkDir, 'Validation'), mode = 0o777) # Path: %Workdir%/Validation

# 再创建训练集文件夹与验证集文件夹的子文件夹
for Subfolder in tqdm(os.listdir(WorkDir)): # 遍历工作目录下的子文件夹并让进度可视化
    if os.path.basename(Subfolder) != 'Training' and os.path.basename(Subfolder) != 'Validation': # 筛选出类文件夹
        os.mkdir(os.path.join(WorkDir, 'Training', Subfolder), mode = 0o777) # Path: %WorkDir%/Training/%Subfolder%
        os.mkdir(os.path.join(WorkDir, 'Validation', Subfolder), mode = 0o777) # Path: %WorkDir%/Validation/%Subfolder%

# 对工作目录名进行分割得到数据集的名字
DataSet_Name = re.split(pattern = '[-_]', string = WorkDir, maxsplit = 0, flags = 0)[0] # 取分割后的第一个字段

## 3. 划分出训练集与验证集并移动文件

In [24]:
'''
一般训练集和测试集的比例为7:3。而为了加强信息保密并更准确地反应模型效能，比较推荐的是将训练集、验证集、测试集的比例划分为6:2:2
'''

ValidationSet_Fraction = 0.3  # 验证集比例

In [ ]:
# 输出统计条目
print('{:^21} {:^21} {:^21}'.format('数据类别', '训练集数据量', '验证集数据量')) # 中间对齐（宽度为21）

# 创建数据表对象
DF = pandas.DataFrame()

# 移动工作目录下的类文件夹至训练集文件夹与验证集文件夹
for Subfolder in tqdm(os.listdir(WorkDir)): # 遍历工作目录下的子文件夹并让进度可视化
    if os.path.basename(Subfolder) != 'Training' and os.path.basename(Subfolder) != 'Validation': # 筛选出类文件夹

        # 获取该类别的文件名并打乱
        Folder_OldPath = os.path.join(WorkDir, Subfolder)
        FileList = os.listdir(Folder_OldPath) #文件名列表
        random.seed(210) # 设置随机种子使函数结果可被复现
        random.shuffle(FileList) # 根据随机种子打乱文件名列表的顺序

        # 划分数据集
        ValidationSet_Content = FileList[:int(len(FileList) * ValidationSet_Fraction)] # 拟移动至验证集文件夹的文件名列表
        TrainingSet_Content   = FileList[int(len(FileList) * ValidationSet_Fraction):] # 拟移动至训练集文件夹的文件名列表

        # 移动文件至验证集文件夹
        for File in ValidationSet_Content:
            File_OldPath = os.path.join(WorkDir, Subfolder, File) # 旧文件路径
            File_NewPath = os.path.join(WorkDir, 'Validation', Subfolder, File) # 新文件路径
            shutil.move(File_OldPath, File_NewPath) # 移动文件

        # 移动文件至训练集文件夹
        for File in TrainingSet_Content:
            File_OldPath = os.path.join(WorkDir, Subfolder, File) # 旧文件路径
            File_NewPath = os.path.join(WorkDir, 'Training', Subfolder, File) # 新文件路径
            shutil.move(File_OldPath, File_NewPath) # 移动文件
        
        # 检查并整理文件信息
        if len(os.listdir(Folder_OldPath)) == 0: # 当旧文件夹中已无文件
            # 删除旧文件夹
            shutil.rmtree(Folder_OldPath)
            # 输出统计信息
            print('{:^21} {:^21} {:^21}'.format(Subfolder, len(TrainingSet_Content), len(ValidationSet_Content)))
            # 保存所得信息到表格中
            DF = DF.append({'数据类别': Subfolder, '训练集数据量': len(TrainingSet_Content), '验证集数据量': len(ValidationSet_Content)}, ignore_index = True)
        else: # 若旧文件夹中仍有文件
            raise AssertionError # 自动引发异常以终止执行

# 移动工作目录下的所有文件至新的数据集文件夹
shutil.move(WorkDir, DataSet_Name + '_Split') # 新文件夹的命名方式为在末尾加上"_Split"

# 统计数据集的文件数量，导出表格为 csv 文件
DF['数据集数据量'] = DF['训练集数据量'] + DF['验证集数据量']
DF.to_csv('数据量统计.csv', index = False)

# 输出数据表
DF